In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
date_format = "%Y-%m-%d"

In [3]:
all_seasons = pd.read_csv('Datasets/all_seasons.csv')
all_seasons = all_seasons[['datetime', 'conditions']]

In [44]:
all_seasons.head()

,datetime,conditions
0,2000-01-01,Partially cloudy
1,2000-01-02,Clear
2,2000-01-03,Clear
3,2000-01-04,Clear
4,2000-01-05,Clear


In [45]:
simplifier = {'Overcast':'no_rain', 'Partially cloudy':'no_rain', 'Clear':'no_rain', 'Rain, Partially cloudy':'rain', 'Rain':'rain', 'Rain, Overcast':'rain'}

all_seasons['condition'] = all_seasons['conditions'].map(simplifier)

In [46]:
all_seasons.head()

,datetime,conditions,condition
0,2000-01-01,Partially cloudy,no_rain
1,2000-01-02,Clear,no_rain
2,2000-01-03,Clear,no_rain
3,2000-01-04,Clear,no_rain
4,2000-01-05,Clear,no_rain


In [47]:
all_seasons = all_seasons[['datetime', 'condition']]

In [48]:
all_seasons.head()

,datetime,condition
0,2000-01-01,no_rain
1,2000-01-02,no_rain
2,2000-01-03,no_rain
3,2000-01-04,no_rain
4,2000-01-05,no_rain


We will refer to rain is 'R' and no rain as 'N'

In [49]:
R_after_R_count = 0.0
N_after_R_count = 0.0

R_after_N_count = 0.0
N_after_N_count = 0.0

In [50]:
all_seasons['condition_shift'] = all_seasons['condition'].shift(-1)

for i in range(len(all_seasons)):
    if all_seasons.loc[i, 'condition'] == 'rain' and all_seasons.loc[i, 'condition_shift'] == 'rain':
        R_after_R_count += 1
    elif all_seasons.loc[i, 'condition'] == 'no_rain' and all_seasons.loc[i, 'condition_shift'] == 'rain':
        N_after_R_count += 1
    elif all_seasons.loc[i, 'condition'] == 'rain' and all_seasons.loc[i, 'condition_shift'] == 'no_rain':
        R_after_N_count += 1
    elif all_seasons.loc[i, 'condition'] == 'no_rain' and all_seasons.loc[i, 'condition_shift'] == 'no_rain':
        N_after_N_count += 1

In [51]:
current_R_total = R_after_R_count + N_after_R_count
current_N_total = R_after_N_count + N_after_N_count

In [54]:
R_after_R_prob = R_after_R_count / current_R_total
N_after_R_prob = N_after_R_count / current_R_total

R_after_N_prob = R_after_N_count / current_N_total
N_after_N_prob = N_after_N_count / current_N_total

In [56]:
# Printing our probabilities for 2x2 transition matrix:
print(R_after_R_prob)
print(N_after_R_prob)
print(R_after_N_prob)
print(N_after_N_prob)

In [57]:
# Checking that each row in the transition matrix adds up to 1:
print(R_after_R_prob + N_after_R_prob)
print(R_after_N_prob + N_after_N_prob)

1.0
1.0


In [58]:
# Creating the transition matrix:
transition_name = [['RR', 'RN'], ['RN', 'NN']]
transition_matrix = [[R_after_R_prob, N_after_R_prob], [R_after_N_prob, N_after_N_prob]]

In [59]:
print(transition_matrix)

[[0.46397941680960547, 0.5360205831903945], [0.08712015611931977, 0.9128798438806802]]
